## Test mlflow setup

- Make sure you have a mlflow instance running - refer to [README.md](./README.md)

In [1]:
import os

import mlflow

### Create mlflow experiment + test parameter/metric logging

In [7]:
mlflow_uri = "http://localhost:5000" # change to your uri + port

mlflow.set_tracking_uri(mlflow_uri)

exp_id = mlflow.set_experiment("mlflow_test1")

mlflow.log_param("param1", "value1")
mlflow.log_param("param2", "value2")

for i in range(10):
    mlflow.log_metric("metric1", value=1/(i+1), step=i)

mlflow.end_run()

- Navigate to e.g. "http://localhost:5000" -> experiments
- Check if a new experiment was created "mlflow_test1" with parameters (param1, param2) and metric1

#### Test artifact logging with s3 backend

- Important concepts
 - the client needs to be aware of the s3 credentials + s3 host/endpoint - therefore we set the corresponding env vars based on the env.sh (here as an example)
 - The bucket name + folder is defined within mlflow server backend settings - if not overwritten/defined otherwise
 - The mlflow server backend only needs the credentials to access uploaded/logged artifacts from within the ui (e.g. http://localhost:5000)
- There are other options - refer to https://www.mlflow.org/docs/latest/tracking.html#artifact-stores

In [3]:
# set the env variables - for s3 host + credentials
s3_vars = ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "MLFLOW_S3_ENDPOINT_URL"]
with open("env.sh", "r") as env_file:
    for line in env_file:
        if line.startswith("export "):
            env_var, value = line[len("export "):].replace("\n", "").split("=")
            value = value.split(" ")[0]

            if env_var in s3_vars:
                os.environ[env_var] = value

exp_id = mlflow.set_experiment("mlflow_test_artifacts1")

mlflow.log_artifact("./test-file.pdf")

mlflow.end_run()

- After logging navigate to e.g. "http://localhost:5000" -> experiments
- Within the created experiment "mlflow_test_artifacts1" there should be an artifact "test-file.pdf".
- Path will look like something "s3://yourbucketname/yourfoldername/2/<UUID>/test-file.pdf"